# Quick EDA

In [10]:
import pandas as pd
import re

# ── Load data ─────────────────────────────────────────────────────────────────
df = pd.read_csv('../Data/spotify_reviews.csv')


# ── Basic text features ───────────────────────────────────────────────────────
df['text_len']      = df['Content'].str.len()
df['word_count']    = df['Content'].str.split().str.len()
df['has_non_ascii'] = df['Content'].str.contains(r'[^\x00-\x7F]', regex=True)

# ── Descriptive stats ─────────────────────────────────────────────────────────
print("=" * 50)
print("📊 Text Statistics")
print("=" * 50)
print(df[['text_len', 'word_count']].describe().round(2))

# ── Noise ratio ───────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("⚠️  Noise / Non-ASCII ratio")
print("=" * 50)
print(f"  Rows with emojis / non-ASCII chars: "
    f"{df['has_non_ascii'].sum()} "
    f"({df['has_non_ascii'].mean()*100:.2f} %)")

# ── Top 10 words ──────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("🔤 Top 10 Most Common Words")
print("=" * 50)
top_words = (
    df['Content']
    .str.lower()
    .str.replace(r'[^\w\s]', '', regex=True)   # strip punctuation first
    .str.split()
    .explode()
    .value_counts()
    .head(10)
    .reset_index()
)
top_words.columns = ['word', 'count']
print(top_words.to_string(index=False))

# ── Quirks list ───────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("🔍 Quirks / Data Quality Flags")
print("=" * 50)
empty_rows      = df['Content'].isna().sum()
blank_rows      = (df['Content'].str.strip() == '').sum()
duplicate_rows  = df['Content'].duplicated().sum()
very_short      = (df['word_count'] < 3).sum()
very_long       = (df['word_count'] > df['word_count'].quantile(0.99)).sum()

quirks = {
    "Missing / NaN texts"       : empty_rows,
    "Blank texts"               : blank_rows,
    "Duplicate texts"           : duplicate_rows,
    "Very short texts (< 3 w)"  : very_short,
    "Very long texts (> 99 pct)": very_long,
    "Non-ASCII / emoji texts"   : df['has_non_ascii'].sum(),
}
for label, val in quirks.items():
    flag = "🚩" if val > 0 else "✅"
    print(f"  {flag}  {label:<35}: {val}")

# ── 3 raw samples ─────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("📝 3 Random Raw Samples")
print("=" * 50)
for i, sample in enumerate(df['Content'].sample(3, random_state=42).tolist(), 1):
    print(f"\n  [{i}] {sample}")

📊 Text Statistics
       text_len  word_count
count    100.00      100.00
mean     117.02       21.30
std      108.00       20.11
min       24.00        3.00
25%       43.00        8.00
50%       78.50       13.50
75%      134.50       24.25
max      500.00      106.00

⚠️  Noise / Non-ASCII ratio
  Rows with emojis / non-ASCII chars: 17 (17.00 %)

🔤 Top 10 Most Common Words
   word  count
     to     74
    the     70
    app     63
premium     48
   this     46
     is     43
    and     35
      i     33
spotify     30
    now     29

🔍 Quirks / Data Quality Flags
  ✅  Missing / NaN texts                : 0
  ✅  Blank texts                        : 0
  ✅  Duplicate texts                    : 0
  ✅  Very short texts (< 3 w)           : 0
  🚩  Very long texts (> 99 pct)         : 1
  🚩  Non-ASCII / emoji texts            : 17

📝 3 Random Raw Samples

  [1] u just make everything paid trash app

  [2] I really love this app this app always give me best jazz to Jinja my self

  [3] Wher

In [11]:
import string

# Extract punctuation intensity BEFORE cleaning
df['exclamation_count'] = df['Content'].str.count(r'!')
df['question_count'] = df['Content'].str.count(r'\?')
df['has_repeated_punct'] = df['Content'].str.contains(r'[!?]{2,}', regex=True)




# Text Normalization

In [12]:
%pip install contractions
import unicodedata
import contractions

def normalize(text):
    # Unicode normalize: é→e, etc.
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    # Expand contractions: can't→cannot, won't→will not, etc.
    text = contractions.fix(text)
    # Case fold
    return text.lower()

df['normalized'] = df['Content'].apply(normalize)

# ── Preview ───────────────────────────────────────────────────────────────────
print("=" * 50)
print("🔤 Normalization Preview (3 samples)")
print("=" * 50)
for orig, norm in zip(df['Content'].head(3), df['normalized'].head(3)):
    print(f"\n  Original  : {orig}")
    print(f"  Normalized: {norm}")

Note: you may need to restart the kernel to use updated packages.
🔤 Normalization Preview (3 samples)

  Original  : Spotify is very good, got many songs and suggests new ones but I've to say the app was way good before than now. Because there are lot of ads showing up. Pls don't ask us to try premium. No I don't want to. Please try to reduce the ads it would be okay if we get 2 to 3 ads in between after listening to 5 or 6 songs. It says"watch this 30 sec ad to listen to songs for 30 mins ad free" but what we get is five 30 sec ads continuously and after listening to 2 songs we again get ads. It's irritating.
  Normalized: spotify is very good, got many songs and suggests new ones but i have to say the app was way good before than now. because there are lot of ads showing up. pls do not ask us to try premium. no i do not want to. please try to reduce the ads it would be okay if we get 2 to 3 ads in between after listening to 5 or 6 songs. it says"watch this 30 sec ad to listen to song

In [13]:
import re

# ── Regex cleaning patterns ───────────────────────────────────────────────────
PATTERNS = [
    (r'https?://\S+',                                         ''),   # URLs
    (r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',''),   # Emails
    (r'@\w+',                                                 ''),   # Handles
    (r'<.*?>',                                                ''),   # HTML tags
    (r'\s+',                                                  ' '),  # Collapse whitespace
]

def regex_clean(text: str) -> str:
    for pattern, replacement in PATTERNS:
        text = re.sub(pattern, replacement, text)
    return text.strip()


df['cleaned'] = df['normalized'].apply(regex_clean)


def remove_punctuation(text: str) -> str:
    return text.translate(str.maketrans('', '', string.punctuation))

df['cleaned'] = df['normalized'].apply(lambda x: remove_punctuation(x))

# ── Preview ───────────────────────────────────────────────────────────────────
print("=" * 60)
print("🔍 Regex Cleaning Preview (3 samples)")
print("=" * 60)
for norm, cleaned in zip(df['normalized'].head(3), df['cleaned'].head(3)):
    print(f"\n  Normalized : {norm}")
    print(f"  Cleaned    : {cleaned}")

🔍 Regex Cleaning Preview (3 samples)

  Normalized : spotify is very good, got many songs and suggests new ones but i have to say the app was way good before than now. because there are lot of ads showing up. pls do not ask us to try premium. no i do not want to. please try to reduce the ads it would be okay if we get 2 to 3 ads in between after listening to 5 or 6 songs. it says"watch this 30 sec ad to listen to songs for 30 mins ad free" but what we get is five 30 sec ads continuously and after listening to 2 songs we again get ads. it is irritating.
  Cleaned    : spotify is very good got many songs and suggests new ones but i have to say the app was way good before than now because there are lot of ads showing up pls do not ask us to try premium no i do not want to please try to reduce the ads it would be okay if we get 2 to 3 ads in between after listening to 5 or 6 songs it sayswatch this 30 sec ad to listen to songs for 30 mins ad free but what we get is five 30 sec ads continuo